## <span style=color:blue>Fetching the more-or-less central lat lon for each county/state pair of interest in our ML pipeline    </span>


In [1]:
# This useful if I want to give unique names to directories or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime

<span style=color:blue>The year_state_county_yield.csv file holds all of the year-county-state triples of interest along with total soybean yield.</span>

In [3]:
import pandas as pd

archives_dir = './Archives/'
file = 'd_KENSAS_WHEAT'

df = pd.read_csv(archives_dir + file)
print('number of rows in csv cleaned for ML: ', len(df))
print()
print(df.head())


df1 = df[['state_name','county_name']].drop_duplicates()
print('\nNumber of state-county pairs is: ', len(df1))


number of rows in csv cleaned for ML:  2342

   year state_name county_name  yield
0  2007     KANSAS    CHEYENNE   47.0
1  2006     KANSAS    CHEYENNE   20.0
2  2005     KANSAS    CHEYENNE   21.0
3  2004     KANSAS    CHEYENNE   18.0
4  2003     KANSAS    CHEYENNE   32.0

Number of state-county pairs is:  105


<span style=color:blue>The function geocode_county defined below won't work on "DU PAGE" county in Illinois.  But it does work on "DUPAGE".  So, changing the name in both df and df1 </span>

In [50]:
#### No need for our dataset ####

# # including the "or" (i.e., "|") so that I can this cell is eidenpotent
# index = df.index[(df['county_name'] == 'DU PAGE') | (df['county_name'] == 'DUPAGE')].tolist()
# print(index)
# for ind in index:
#     df.at[ind, 'county_name'] = 'DUPAGE'
#     print(df.at[ind, 'county_name'])

# index1 = df1.index[(df1['county_name'] == 'DU PAGE') | (df1['county_name'] == 'DUPAGE')].tolist()
# print(index1)
# for ind in index1:
#     df1.at[ind, 'county_name'] = 'DUPAGE'
#     print(df1.at[ind, 'county_name'])


[279, 280, 281, 282, 283]
DUPAGE
DUPAGE
DUPAGE
DUPAGE
DUPAGE
[279]
DUPAGE


<span style=color:blue>Using geopy to fetch lon-lat for (approx center of) each county.  This cell takes a while to run</span>

In [6]:
from geopy.geocoders import Nominatim

# Geocoding function to retrieve coordinates for a county
def geocode_county(state, county):
    geolocator = Nominatim(user_agent="county_geocoder")
    location = geolocator.geocode(county + ", " + state + ", USA")
    if location:
        return location.longitude, location.latitude
    else:
        print('no lat-lon found for ', state, county)
        return None, None
        
        
df1['lon'] = df1.apply(lambda x: geocode_county(x['state_name'], x['county_name'])[0], axis=1)
df1['lat'] = df1.apply(lambda x: geocode_county(x['state_name'], x['county_name'])[1], axis=1)

print(df1.head())    
print()

print('lon-lat for KANSAS-BROWN is: ', geocode_county('KANSAS', 'BROWN'))


   state_name county_name         lon        lat
0      KANSAS    CHEYENNE -101.757549  39.795580
5      KANSAS     DECATUR -100.472769  39.794053
10     KANSAS      GRAHAM  -99.898062  39.340620
15     KANSAS      NORTON  -99.910003  39.794470
20     KANSAS     RAWLINS -101.099472  39.790480

lon-lat for KANSAS-BROWN is:  (-95.5819952, 39.824223)


<span style=color:blue>Archiving df1 for later use </span>

In [7]:
archives_dir = './Archives/'
filename = 'state_county_lon_lat.csv'
df1.to_csv(archives_dir + filename, index=False)
print('wrote file: ', archives_dir + filename)

wrote file:  ./Archives/state_county_lon_lat.csv
